In [ ]:
############################ HÄMTA ALL DATA ########################
# Start with the statistics 

In [ ]:
# Connect to my database

import logging
import mysql.connector
import requests

# Logging
logging.basicConfig(level=logging.DEBUG, filename='odds_log.log', 
                    format='%(asctime)s - %(levelname)s - %(message)s')

import mysql.connector
import requests

# API-key and url 
API_KEY = 'DettaÄrEjRiktigaAPINyckeln'
API_URL = 'https://v3.football.api-sports.io'

# db connection
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'bertbot123',
    'database': 'football_statistics'
}

# connect to db
db_connection = mysql.connector.connect(**db_config)
cursor = db_connection.cursor()


In [ ]:
############ Start with getting all leagues 

In [ ]:
def fetch_leagues_by_season(season_year):
    logging.info(f"--- Hämta ligor för säsong {season_year} ---")
    headers = {'x-apisports-key': API_KEY}

    try:
        response = requests.get(f"{API_URL}/leagues?season={season_year}", headers=headers)
        if response.status_code != 200:
            logging.error(f"Misslyckades att hämta ligor för säsong {season_year}: {response.status_code} - {response.text}")
            return
        
        leagues = response.json().get('response', [])
        logging.info(f"Antal ligor mottagna för säsong {season_year}: {len(leagues)}")

        # Leagues
        desired_leagues = {
            'Allsvenskan': 'Sweden',
            'Bundesliga': 'Germany',
            'Premier League': 'England',
            'Ligue 1': 'France',
            'Serie A': 'Italy',
            'La Liga': 'Spain'
        }

        for league in leagues:
            league_name = league['league']['name']
            league_country = league['country']['name'] if isinstance(league['country'], dict) else league['country']
            league_id = league['league']['id']
            logo = league['league']['logo']
            flag = league['country']['flag'] if isinstance(league['country'], dict) else None

            if league_name in desired_leagues and league_country == desired_leagues[league_name]:
                logging.info(f"--- Hanterar liga {league_name} från {league_country} för säsong {season_year} ---")

                # Saves teams in database
                try:
                    cursor.execute("""
                        INSERT INTO leagues (league_id, name, country, season, logo, flag)
                        VALUES (%s, %s, %s, %s, %s, %s)
                        ON DUPLICATE KEY UPDATE 
                            name = VALUES(name), 
                            logo = VALUES(logo), 
                            flag = VALUES(flag);
                    """, (league_id, league_name, league_country, season_year, logo, flag))

                    db_connection.commit()  # COMMIT
                    logging.info(f"Liga {league_name} för säsong {season_year} sparades.")

                except mysql.connector.Error as err:
                    logging.error(f"Fel vid insättning av liga {league_name} för säsong {season_year}: {err}")

        
        cursor.execute("SELECT COUNT(*) FROM leagues WHERE season=%s", (season_year,))
        count = cursor.fetchone()[0]
        logging.info(f"Antal ligor i databasen för säsong {season_year}: {count}")

    except Exception as e:
        logging.error(f"Ett fel uppstod vid hämtning av ligor för säsong {season_year}: {e}")

# run functions
fetch_leagues_by_season(2022)
fetch_leagues_by_season(2023)
fetch_leagues_by_season(2024)




In [ ]:
### FETCH THE TEAMS 

In [ ]:
def fetch_teams_for_leagues():
    headers = {'x-apisports-key': API_KEY}

    # Fetch the leagues from database change year for different season
    cursor.execute("SELECT league_id, season FROM leagues WHERE season IN (2024)")
    leagues = cursor.fetchall()

    logging.info(f"Hämtade {len(leagues)} ligor från databasen.")

    for league in leagues:
        league_id, season = league
        logging.info(f"Hämtar lag för liga ID {league_id} och säsong {season}")
        
        try:
            response = requests.get(f"{API_URL}/teams?league={league_id}&season={season}", headers=headers)
            if response.status_code != 200:
                logging.error(f"Misslyckades att hämta lag för liga {league_id} och säsong {season}: {response.status_code}")
                continue
            
            teams = response.json().get('response', [])
            logging.info(f"Hämtade {len(teams)} lag för liga ID {league_id} och säsong {season}")

            for team in teams:
                team_id = team['team']['id']
                team_name = team['team']['name']
                country = team['team']['country']
                founded = team['team'].get('founded')
                logo = team['team'].get('logo')
                venue_name = team['venue'].get('name')
                venue_address = team['venue'].get('address')
                venue_city = team['venue'].get('city')
                venue_capacity = team['venue'].get('capacity')

                # Save the teams in the database
                try:
                    cursor.execute("""
                        INSERT INTO teams (team_id, name, league_id, country, founded, logo, venue_name, venue_address, venue_city, venue_capacity)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                        ON DUPLICATE KEY UPDATE 
                        name=VALUES(name), 
                        founded=VALUES(founded), 
                        logo=VALUES(logo),
                        venue_name=VALUES(venue_name), 
                        venue_address=VALUES(venue_address), 
                        venue_city=VALUES(venue_city), 
                        venue_capacity=VALUES(venue_capacity);
                    """, (team_id, team_name, league_id, country, founded, logo, venue_name, venue_address, venue_city, venue_capacity))
                    
                    db_connection.commit()
                    logging.info(f"Laget {team_name} har sparats i databasen.")

                except mysql.connector.Error as err:
                    logging.error(f"Fel vid insättning av lag {team_name}: {err}")

        except Exception as e:
            logging.error(f"Fel vid hämtning av lag för liga {league_id} och säsong {season}: {e}")

# Run function 
fetch_teams_for_leagues()



In [ ]:
####### Fetch all the matches

In [ ]:
def fetch_matches_for_leagues():
    headers = {'x-apisports-key': API_KEY}

    # Get league and season from database
    cursor.execute("SELECT league_id, season FROM leagues WHERE season IN (2024)")
    leagues = cursor.fetchall()

    logging.info(f"Hämtade {len(leagues)} ligor från databasen.")

    for league_id, season in leagues:
        logging.info(f"Hämtar matcher för liga {league_id} och säsong {season}")

        try:
            response = requests.get(f"{API_URL}/fixtures?league={league_id}&season={season}", headers=headers)
            if response.status_code != 200:
                logging.error(f"Misslyckades att hämta matcher för liga {league_id} och säsong {season}: {response.status_code}")
                continue

            fixtures = response.json().get('response', [])
            logging.info(f"Hämtade {len(fixtures)} matcher för liga {league_id} och säsong {season}")

            for fixture in fixtures:
                match_id = fixture['fixture']['id']
                date = fixture['fixture']['date']
                home_team_id = fixture['teams']['home']['id']
                away_team_id = fixture['teams']['away']['id']
                home_score = fixture['goals']['home']
                away_score = fixture['goals']['away']
                status = fixture['fixture']['status']['short']
                referee = fixture['fixture']['referee']
                venue = fixture['fixture']['venue']['name']

                # save matches in database
                try:
                    cursor.execute("""
                        INSERT INTO matches (match_id, league_id, season, date, home_team_id, away_team_id, home_score, away_score, status, referee, venue)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                        ON DUPLICATE KEY UPDATE 
                        home_score=VALUES(home_score), 
                        away_score=VALUES(away_score), 
                        status=VALUES(status), 
                        referee=VALUES(referee), 
                        venue=VALUES(venue);
                    """, (match_id, league_id, season, date, home_team_id, away_team_id, home_score, away_score, status, referee, venue))
                    
                    db_connection.commit()
                    logging.info(f"Match {match_id} har sparats i databasen.")

                except mysql.connector.Error as err:
                    logging.error(f"Fel vid insättning av match {match_id}: {err}")

        except Exception as e:
            logging.error(f"Fel vid hämtning av matcher för liga {league_id} och säsong {season}: {e}")

# Run the function
fetch_matches_for_leagues()


In [ ]:
############ FETCH TEAM STATISTICS

In [ ]:
def fetch_team_statistics():
    headers = {'x-apisports-key': API_KEY}

    # SQL query to fetch all teams and matches in database
    cursor.execute("""
        SELECT m.match_id, t.team_id 
        FROM matches m
        JOIN teams t ON m.home_team_id = t.team_id OR m.away_team_id = t.team_id
    """)
    teams_matches = cursor.fetchall()

    logging.info(f"Hämtade {len(teams_matches)} match- och lagkombinationer från databasen.")

    for match_id, team_id in teams_matches:
        logging.info(f"Hämtar statistik för lag ID {team_id} i match ID {match_id}")
        
        try:
            response = requests.get(f"{API_URL}/fixtures/statistics?fixture={match_id}&team={team_id}", headers=headers)
            logging.info(f"API response status för lag {team_id} i match {match_id}: {response.status_code}")
            logging.info(f"API response text: {response.text}")

            if response.status_code != 200:
                logging.error(f"Misslyckades att hämta statistik för lag {team_id} i match {match_id}: {response.status_code}")
                continue
            
            statistics = response.json().get('response', [])
            if not statistics:
                logging.warning(f"Ingen statistik hittades för lag {team_id} i match {match_id}.")
                continue

            stats = statistics[0]['statistics']

            # Extrahera statistik och hantera bollinnehav (ta bort procenttecken och konvertera till int)
            possession = next((item['value'] for item in stats if item['type'] == 'Ball Possession'), None)
            if possession:
                possession = int(possession.replace('%', ''))  # Ta bort procenttecken och konvertera till int

            shots_on_goal = next((item['value'] for item in stats if item['type'] == 'Shots on Goal'), None)
            shots_off_goal = next((item['value'] for item in stats if item['type'] == 'Shots off Goal'), None)
            total_shots = next((item['value'] for item in stats if item['type'] == 'Total Shots'), None)
            blocked_shots = next((item['value'] for item in stats if item['type'] == 'Blocked Shots'), None)
            corners = next((item['value'] for item in stats if item['type'] == 'Corner Kicks'), None)
            offsides = next((item['value'] for item in stats if item['type'] == 'Offsides'), None)
            fouls = next((item['value'] for item in stats if item['type'] == 'Fouls'), None)
            yellow_cards = next((item['value'] for item in stats if item['type'] == 'Yellow Cards'), None)
            red_cards = next((item['value'] for item in stats if item['type'] == 'Red Cards'), None)
            passes = next((item['value'] for item in stats if item['type'] == 'Passes'), None)
            passes_completed = next((item['value'] for item in stats if item['type'] == 'Passes accurate'), None)
            tackles = next((item['value'] for item in stats if item['type'] == 'Tackles'), None)

            # Spara statistik i databasen
            try:
                cursor.execute("""
                    INSERT INTO team_match_statistics (match_id, team_id, possession_percentage, shots_on_goal, shots_off_goal, total_shots, 
                    blocked_shots, corners, offsides, fouls, yellow_cards, red_cards, passes, passes_completed, tackles)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    ON DUPLICATE KEY UPDATE 
                    possession_percentage=VALUES(possession_percentage), 
                    shots_on_goal=VALUES(shots_on_goal), 
                    shots_off_goal=VALUES(shots_off_goal), 
                    total_shots=VALUES(total_shots), 
                    blocked_shots=VALUES(blocked_shots),
                    corners=VALUES(corners), 
                    offsides=VALUES(offsides), 
                    fouls=VALUES(fouls), 
                    yellow_cards=VALUES(yellow_cards), 
                    red_cards=VALUES(red_cards), 
                    passes=VALUES(passes), 
                    passes_completed=VALUES(passes_completed), 
                    tackles=VALUES(tackles);
                """, (match_id, team_id, possession, shots_on_goal, shots_off_goal, total_shots, blocked_shots, corners, offsides, fouls, yellow_cards, red_cards, passes, passes_completed, tackles))
                
                db_connection.commit()
                logging.info(f"Lagstatistik för lag {team_id} i match {match_id} har sparats i databasen.")

            except mysql.connector.Error as err:
                logging.error(f"Fel vid insättning av lagstatistik för lag {team_id} i match {match_id}: {err}")

        except Exception as e:
            logging.error(f"Fel vid hämtning av lagstatistik för lag {team_id} i match {match_id}: {e}")

# Kör funktionen för att hämta lagstatistik
fetch_team_statistics()



In [ ]:
##### fetch player statistics

In [ ]:
def fetch_players_and_statistics(season_year):
    headers = {'x-apisports-key': API_KEY}

    # fetch all teams from database
    cursor.execute("SELECT team_id FROM teams")
    teams = cursor.fetchall()

    logging.info(f"Hämtade {len(teams)} lag från databasen för säsong {season_year}.")

    total_teams = len(teams)
    processed_teams = 0

    for team in teams:
        team_id = team[0]
        processed_teams += 1
        logging.info(f"Hämtar spelare för lag ID {team_id} ({processed_teams}/{total_teams}) för säsong {season_year}")

        # check if team played any matches this season
        fixtures_response = requests.get(f"{API_URL}/fixtures?team={team_id}&season={season_year}", headers=headers)
        fixtures = fixtures_response.json().get('response', [])
        
        if not fixtures:
            logging.info(f"Lag ID {team_id} spelade inga matcher under säsong {season_year}. Hoppar över spelare.")
            continue  

        try:
            # fetch players for team if team played
            players_response = requests.get(f"{API_URL}/players?team={team_id}&season={season_year}", headers=headers)
            if players_response.status_code != 200:
                logging.error(f"Misslyckades att hämta spelare för lag {team_id} för säsong {season_year}: {players_response.status_code}")
                continue

            players = players_response.json().get('response', [])
            logging.info(f"Hämtade {len(players)} spelare för lag ID {team_id} för säsong {season_year}")

            no_stats_count = 0  

            for player_data in players:
                player = player_data['player']
                statistics = player_data['statistics'][0]['games'] if player_data['statistics'] else {}

                player_id = player['id']
                player_name = player['name']
                age = player.get('age')
                nationality = player.get('nationality')
                position = statistics.get('position')  
                height = player.get('height')
                weight = player.get('weight')

                # stats
                appearances = statistics.get('appearences', 0)  
                goals = player_data['statistics'][0]['goals'].get('total', 0)
                assists = player_data['statistics'][0]['goals'].get('assists', 0)
                yellow_cards = player_data['statistics'][0]['cards'].get('yellow', 0)
                red_cards = player_data['statistics'][0]['cards'].get('red', 0)

                # Save players
                try:
                    cursor.execute("""
                        INSERT INTO players (player_id, team_id, name, age, nationality, position, height, weight, appearances, goals, assists, yellow_cards, red_cards) 
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                        ON DUPLICATE KEY UPDATE 
                        name=VALUES(name), 
                        age=VALUES(age), 
                        nationality=VALUES(nationality), 
                        position=VALUES(position),
                        height=VALUES(height),
                        weight=VALUES(weight),
                        appearances=VALUES(appearances),
                        goals=VALUES(goals),
                        assists=VALUES(assists),
                        yellow_cards=VALUES(yellow_cards),
                        red_cards=VALUES(red_cards);
                    """, (player_id, team_id, player_name, age, nationality, position, height, weight, appearances, goals, assists, yellow_cards, red_cards))

                    db_connection.commit()

                except mysql.connector.Error as err:
                    logging.error(f"Fel vid insättning av spelare {player_name} (ID: {player_id}): {err}")

            # Logg how many players had missing stats
            if no_stats_count > 0:
                logging.info(f"{no_stats_count} spelare i lag ID {team_id} hade ingen matchstatistik.")

        except Exception as e:
            logging.error(f"Fel vid hämtning av spelare för lag ID {team_id}: {e}")

# change year to fetch another season 
fetch_players_and_statistics(2024)



In [ ]:
############## Fetch all odds

In [ ]:
import requests
import mysql.connector
from datetime import datetime
import logging

logging.basicConfig(level=logging.DEBUG, filename='odds_log.log', 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Din API-nyckel från Odds-API
api_key = "fejknyckel"

# Anslut till din MySQL-databas
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="bertbot123",
    database="football_statistics"
)

cursor = conn.cursor()

In [ ]:
############################################## Function to fetch odds for season 2024. 
# Change needed parts to change league 
import requests
import logging
from datetime import datetime
import mysql.connector


# Configure logging
logging.basicConfig(level=logging.INFO, filename='odds_log.log',
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Allowed bookmakers to be considered (only Unibet) since took too many API-calls with more
allowed_bookmakers = ["Unibet"]


# Function to fetch match dates, date is set for this season 
def fetch_match_dates_from_db(league_id):
    query = """
    SELECT match_id, date 
    FROM matches 
    WHERE season = 2024 AND league_id = %s AND date >= '2024-04-01' AND date <= '2024-12-01'
    """  
    cursor.execute(query, (league_id,))
    matches = cursor.fetchall()
    return matches

# Function to check if odds and team names for the match already exist
def check_existing_data(match_id):
    query = """
    SELECT home_team_name, away_team_name 
    FROM odds_new 
    WHERE match_id = %s AND bookmaker = 'Unibet'
    """
    cursor.execute(query, (match_id,))
    result = cursor.fetchone()
    if result:
        home_team_name, away_team_name = result
        # Check if team names are both present
        return home_team_name is not None and away_team_name is not None
    return False

# Function to fetch odds from the API for a specific match
def fetch_odds_for_match(league_key, event_date):
    event_timestamp = f"{event_date}T12:00:00Z"
    logging.info(f"Fetching odds for {league_key} on {event_timestamp}")
    
    url = f"https://api.the-odds-api.com/v4/sports/{league_key}/odds-history"
    params = {
        'apiKey': api_key,
        'date': event_timestamp,
        'regions': 'eu',
        'markets': 'h2h'
    }
    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        logging.error(f"Error fetching odds: {response.status_code} - {response.text}")
        return None


##################################################################################################################################
##################################################################################################################################

# Function to save odds and team information to the database
# See cell below for other options to change to other league
def save_odds_to_db(odds_data, league_key):
    leagues = {
        'soccer_sweden_allsvenskan': 113  # Allsvenskan ID
    }
    
    insert_query = """
    INSERT INTO odds_new (match_id, bookmaker, home_odds, draw_odds, away_odds, league_id, season, event_date, home_team_name, away_team_name)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE 
        home_odds=VALUES(home_odds), 
        draw_odds=VALUES(draw_odds), 
        away_odds=VALUES(away_odds),
        home_team_name=VALUES(home_team_name),
        away_team_name=VALUES(away_team_name),
        league_id=VALUES(league_id), 
        season=VALUES(season), 
        event_date=VALUES(event_date)
    """
    
    for match in odds_data.get('data', []):
        try:
            if 'bookmakers' in match and len(match['bookmakers']) > 0:
                for bookmaker_data in match['bookmakers']:
                    if bookmaker_data['title'].lower() == 'unibet':
                        match_id = match['id']
                        home_team = match['home_team']
                        away_team = match['away_team']
                        h2h_odds = bookmaker_data['markets'][0]['outcomes']
                        home_odds = h2h_odds[0]['price']
                        draw_odds = h2h_odds[1]['price']
                        away_odds = h2h_odds[2]['price']
                        league_id = leagues[league_key]  
                        season = 2024
                        event_date = datetime.strptime(match['commence_time'], "%Y-%m-%dT%H:%M:%SZ").date()
                        
                        # Log the data being saved
                        logging.info(f"Attempting to save match: {match_id}, teams: {home_team} vs {away_team}")

                        cursor.execute(insert_query, (match_id, "Unibet", home_odds, draw_odds, away_odds, league_id, season, event_date, home_team, away_team))
                        conn.commit()
                        logging.info(f"Odds successfully saved for match {match_id}")
        except Exception as e:
            logging.error(f"Error saving odds to database: {e}")

# Main function to run the fetching and saving process change league_key and league_id to fetch another league
def fetch_odds():
    league_key = 'soccer_sweden_allsvenskan'
    league_id = 113  # Allsvenskan
    logging.info(f"Fetching matches for Allsvenskan")
    
    matches = fetch_match_dates_from_db(league_id)
    
    for match_id, match_date in matches:
        if check_existing_data(match_id):
            logging.info(f"Odds and team names already exist for match {match_id}. Skipping.")
            continue
        
        odds_data = fetch_odds_for_match(league_key, match_date)
        if odds_data:
            save_odds_to_db(odds_data, league_key)

# Run the function
fetch_odds()

# Close the database connection
conn.commit()
conn.close()


In [ ]:
# League keys 
    'soccer_italy_serie_a': 135,# done 
    'soccer_spain_la_liga': 140,# done
    'soccer_sweden_allsvenskan': 113,
    'soccer_epl': 39,# done 
    'soccer_germany_bundesliga': 78, # done
    'soccer_france_ligue_one': 61#
